# 01 - Ingestion Kafka

Producer qui récupère les données de l'API OpenSky Network et les envoie à Kafka.

## Configuration

In [3]:
%pip install -r requirements.txt

import threading
import time
import json
import requests
from kafka import KafkaProducer
from datetime import datetime
import os
from dotenv import load_dotenv

load_dotenv()

KAFKA_BOOTSTRAP = os.getenv("KAFKA_BOOTSTRAP", "kafka1:9092")
TOPIC_NAME = os.getenv("TOPIC_NAME", "opensky-data")

print(f"✅ Kafka: {KAFKA_BOOTSTRAP}")
print(f"✅ Topic: {TOPIC_NAME}")

Ignoring kafka-python: markers 'python_version < "3.12"' don't match your environment
  Cloning https://github.com/dpkp/kafka-python.git to /tmp/pip-install-ab3sd88a/kafka-python_a927a2df6bbf4585802507b865b18642
  Running command git clone --filter=blob:none --quiet https://github.com/dpkp/kafka-python.git /tmp/pip-install-ab3sd88a/kafka-python_a927a2df6bbf4585802507b865b18642
  Resolved https://github.com/dpkp/kafka-python.git to commit f8a91ec4d57caa5440d496c5dab937b7c4498e9a
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.3/14.3 MB 2.3 MB/s  0:00:06 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.4/11.4 MB 4.6 MB/s  0:00:02 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.9/13.9 MB 1.2 MB/s  0:00:11m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 1.5 MB/s  0:00:05m0:00:0100:010m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

## Producer

In [ ]:
stop_producer = False

def run_producer():
    """Récupère les données OpenSky et les envoie à Kafka."""
    print(f"🚀 Producer démarré...")
    
    try:
        producer = KafkaProducer(
            bootstrap_servers=KAFKA_BOOTSTRAP,
            value_serializer=lambda v: json.dumps(v).encode('utf-8')
        )
    except Exception as e:
        print(f"❌ Erreur connexion Kafka: {e}")
        return

    api_url = "https://opensky-network.org/api/states/all"

    while not stop_producer:
        try:
            response = requests.get(api_url)
            if response.status_code == 200:
                data = response.json()
                states = data.get('states') or []
                timestamp = data['time']

                for s in states:
                    record = {
                        "time": timestamp, "icao24": s[0],
                        "callsign": s[1].strip() if s[1] else None,
                        "origin_country": s[2], "time_position": s[3],
                        "last_contact": s[4], "longitude": s[5], "latitude": s[6],
                        "baro_altitude": s[7], "on_ground": s[8], "velocity": s[9],
                        "true_track": s[10], "vertical_rate": s[11],
                        "geo_altitude": s[13], "squawk": s[14], "spi": s[15],
                        "position_source": s[16],
                        "category": s[17] if len(s) > 17 else None
                    }
                    producer.send(TOPIC_NAME, record)
                
                producer.flush()
                print(f"📡 {len(states)} vols envoyés ({datetime.now().strftime('%H:%M:%S')})")
            else:
                print(f"⚠️ API Status: {response.status_code}")
        except Exception as e:
            print(f"⚠️ Erreur: {e}")
        time.sleep(15)
    
    print("🛑 Producer arrêté.")

producer_thread = threading.Thread(target=run_producer, daemon=True)
producer_thread.start()
print("✅ Producer en arrière-plan")

🚀 Producer démarré...✅ Producer en arrière-plan



📡 8755 vols envoyés (14:10:26)
📡 8789 vols envoyés (14:10:46)
📡 8793 vols envoyés (14:11:10)
📡 8818 vols envoyés (14:11:34)
📡 8825 vols envoyés (14:11:59)
📡 8806 vols envoyés (14:12:25)
📡 8813 vols envoyés (14:12:53)
🛑 Producer arrêté.


## Arrêt du Producer

In [16]:
stop_producer = True
print("🛑 Arrêt demandé")

🛑 Arrêt demandé
